In [ ]:
import numpy as np                           #complex matrix multiplication
from keras import layers                     
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K


In [94]:
data = pd.read_csv('hindi.csv')
data["character"] = data["character"].replace('character_01_ka',1)
data["character"] = data["character"].replace('character_02_kha',2)
data["character"] = data["character"].replace('character_03_ga',3)
data["character"] = data["character"].replace('character_04_gha',4)
data["character"] = data["character"].replace('character_05_kna',5)
data["character"] = data["character"].replace('character_06_cha',6)
data["character"] = data["character"].replace('character_07_chha',7)
data["character"] = data["character"].replace('character_08_ja',8)
data["character"] = data["character"].replace('character_09_jha',9)
data["character"] = data["character"].replace('character_10_yna',10)
data["character"] = data["character"].replace('character_11_taamatar',11)
data["character"] = data["character"].replace('character_12_thaa',12)
data["character"] = data["character"].replace('character_13_daa',13)
data["character"] = data["character"].replace('character_14_dhaa',14)
data["character"] = data["character"].replace('character_15_adna',15)
data["character"] = data["character"].replace('character_16_tabala',16)
data["character"] = data["character"].replace('character_17_tha',17)
data["character"] = data["character"].replace('character_18_da',18)
data["character"] = data["character"].replace('character_19_dha',19)
data["character"] = data["character"].replace('character_20_na',20)
data["character"] = data["character"].replace('character_21_pa',21)
data["character"] = data["character"].replace('character_22_pha',22)
data["character"] = data["character"].replace('character_23_ba',23)
data["character"] = data["character"].replace('character_24_bha',24)
data["character"] = data["character"].replace('character_25_ma',25)
data["character"] = data["character"].replace('character_26_yaw',26)
data["character"] = data["character"].replace('character_27_ra',27)
data["character"] = data["character"].replace('character_28_la',28)
data["character"] = data["character"].replace('character_29_waw',29)
data["character"] = data["character"].replace('character_30_motosaw',30)
data["character"] = data["character"].replace('character_31_petchiryakha',31)
data["character"] = data["character"].replace('character_32_patalosaw',32)
data["character"] = data["character"].replace('character_33_ha',33)
data["character"] = data["character"].replace('character_34_chhya',34)
data["character"] = data["character"].replace('character_35_tra',35)
data["character"] = data["character"].replace('character_36_gya',36)
data["character"] = data["character"].replace('digit_0',37)
data["character"] = data["character"].replace('digit_1',38)
data["character"] = data["character"].replace('digit_2',39)
data["character"] = data["character"].replace('digit_3',40)
data["character"] = data["character"].replace('digit_4',41)
data["character"] = data["character"].replace('digit_5',42)
data["character"] = data["character"].replace('digit_6',43)
data["character"] = data["character"].replace('digit_7',44)
data["character"] = data["character"].replace('digit_8',45)
data["character"] = data["character"].replace('digit_9',46)

print(data['character'].unique())


dataset = np.array(data)
np.random.shuffle(dataset)
X = dataset[:, 0:1024]
Y = dataset[:,1024:1025]
print(X.shape)
print(Y.shape)
print(Y)


X_train = X[0:90000]
X_train = X_train/255.
X_test = X[90000:92001,:]
X_test = X_test/255.

Y_train = Y[0:90000,:]
Y_train = Y_train.T
Y_test = Y[90000:92000,:]
Y_test =  Y_test.T



[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]
(92000, 1024)
(92000, 1)
[[26]
 [26]
 [38]
 ...
 [44]
 [29]
 [27]]


In [95]:
print('training examples :', X_train.shape[0])
print('test examples :', X_test.shape[0])
print('X_train shape',X_train.shape)
print('X_test shape',X_test.shape)
print('Y_train shape',Y_train.shape)
print('Y_test shape',Y_test.shape)
print(data['character'].unique())


training examples : 90000
test examples : 2000
X_train shape (90000, 1024)
X_test shape (2000, 1024)
Y_train shape (1, 90000)
Y_test shape (1, 2000)
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46]


In [99]:
image_x = 32
image_y = 32   #size of image 32*32


train_y = np_utils.to_categorical(Y_train)
test_y = np_utils.to_categorical(Y_test)

train_y = train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1],test_y.shape[2])

X_train = X_train.reshape(X_train.shape[0],image_x,image_y,1)
X_test = X_test.reshape(X_test.shape[0],image_x,image_y,1)


print('X_train shape:'+str(X_train.shape))
print('Y_train shape:'+str(train_y.shape))

X_train shape:(90000, 32, 32, 1)
Y_train shape:(90000, 47)


In [102]:
#building a model

def keras_model(image_x,image_y):
    num_of_classes = 47
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5,5), input_shape=(image_x, image_y, 1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding ='same'))
    model.add(Conv2D(64, (5,5), activation ='relu'))
    model.add(MaxPooling2D(pool_size=(5,5), strides=(5,5), padding='same'))
    model.add(Flatten())  #matrix to vector
    model.add(Dense(num_of_classes, activation = 'softmax')) #last layer
    model.compile(loss='categorical_crossentropy', optimizer ='adam', metrics = ['accuracy'])
    filepath = "devanagri.h5"
    checkpoint1 = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only = True, mode='max')
    callbacks_list = [checkpoint1]
    return model,callbacks_list
    
    


In [108]:
model, callbacks_list = keras_model(image_x,image_y)
model.fit(X_train,train_y, validation_data = (X_test,test_y), epochs = 8 , batch_size = 64, callbacks = callbacks_list)
scores = model.evaluate(X_test,test_y, verbose = 0)
print("CNN error :%.2f%%" %(100-scores[1]*100))
print_summary(model)   

Train on 90000 samples, validate on 2000 samples
Epoch 1/8
90000/90000 [==============================] - 127s 1ms/step - loss: 0.6853 - acc: 0.8101 - val_loss: 0.2601 - val_acc: 0.9260

Epoch 00001: val_acc improved from -inf to 0.92600, saving model to devanagri.h5
Epoch 2/8
90000/90000 [==============================] - 119s 1ms/step - loss: 0.1892 - acc: 0.9432 - val_loss: 0.1680 - val_acc: 0.9465

Epoch 00002: val_acc improved from 0.92600 to 0.94650, saving model to devanagri.h5
Epoch 3/8
90000/90000 [==============================] - 122s 1ms/step - loss: 0.1203 - acc: 0.9641 - val_loss: 0.1743 - val_acc: 0.9495

Epoch 00003: val_acc improved from 0.94650 to 0.94950, saving model to devanagri.h5
Epoch 4/8
90000/90000 [==============================] - 126s 1ms/step - loss: 0.0890 - acc: 0.9728 - val_loss: 0.1393 - val_acc: 0.9575

Epoch 00004: val_acc improved from 0.94950 to 0.95750, saving model to devanagri.h5
Epoch 5/8
90000/90000 [==============================] - 122s 1ms/

In [109]:
model.save('devanagri.h5')